# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Jared Vitug
- Miguel Morales
- Phat Ly
- Kevin Mach

# Group Members IDs

- A92083122
- A########
- A########
- A12647584

# Research Question

Can we predict a current winning nba bracket from regular season matchup statistics collected over the last 10 years?
In a best-of-seven NBA playoff series, two teams will matchup against each other consecutively at least four times. This is unlike the regular season schedule in any way, where teams in the same division play at most four times but over the span of an 82 game season. However, we believe that from regular season statistics gathered from the eight current playoff matchups, over the span of the last ten years, we can predict an accurate playoff bracket for the current season’s playoffs. We will look at data such as each team’s winning percentage, number of assists, total offensive rebounds, number of roster changes, etc. to create a model that will tell us which team will win in a playoff series. We will take that model and use it to pick the winners of our bracket.


## Background and Prior Work

With NBA playoffs in full effect, we knew we wanted to research something relative to basketball that was also fun.  We all know sports are near impossible to predict and basketball is no exception - or is it? We began to wonder what if, given a playoff bracket, we could predict each round of the playoffs and thereby figure out who would win the championship.

Each of us watches basketball and knows a few plays can define a game.  Injuries happen, human error occurs, and upsets take place.  However, in the world of sports betting, none of this matters - only numbers matter.  That’s why we decided to find the best combination of regular season matchup stats (between two teams pitted against each other in the playoffs) that can successfully predict a series outcome.

When it comes to the individual statistics, we know that the team which wins the turnover battle usually comes out on top, as does the team with more rebounds and shooting percentage.  However, with all the different play-styles in today’s NBA (iso-ball, backcourt dominance, frontcourt dominance) we want to find out if there are other team stats that can help us predict winners, such as three pointers taken, team fouls, timeouts used, etc.


https://towardsdatascience.com/predicting-nba-winning-percentage-in-upcoming-season-using-linear-regression-f8687d9c0418
https://github.com/COGS108/FinalProjects-Wi18/blob/master/001-FinalProject.ipynb
The above link is to a similar project where someone took 40 years of basketball data from basketball-reference.com and trained the data to predict NBA winning percentage.  The article says that “average age of the players, margin of victory, number of points scored, number of returning players, and number of blocks” were some of the statistics included, which gives us some idea as to which statistics to pull.   The study’s analysis discusses how wrong the model was in over-predicting winning seasons and underpredicting losing seasons with the reason being injuries, trades, and retirements - all things which are hard to predict and difficult to assess from a mathematical perspective.

# Hypothesis


We hypothesize that it is possible to predict the winner of a 7-game playoff series between two NBA teams by analyzing their regular season matchup statistics.
By narrowing our data to historical team matchups, we can examine specific factors that may be pertinent in explaining why one team may have the competitive edge. There are many accessible statistics for every NBA game, and this data can be utilized to determine if one franchise may have a playstyle better suited for a particular opponent. Features such as the record of the matchups (giving more weight to recent wins), number of possessions, offensive rebounds, total assists, turnovers, etc. are all telling of how advantageous a team is over the other. Although teams change over time due to player, coach, and management contracts, we will attempt to take those conditions into account by feature engineering, as winning teams tend to have less structure change. Additionally, we can favor more recent data as it will be more telling of the current matchup. Therefore, with a catalog of in depth matchup statistics, we hypothesize that we can create a strong dataset for our model to predict a correct playoff bracket.


# Dataset(s)

Dataset Name: NBA Advanced Stat
Dataset Source : https://stats.nba.com/
Dataset Link  : https://stats.nba.com/teams/traditional/?sort=W_PCT&dir=-1&Season=2018-19&SeasonType=Regular%20Season  	
Number of datasets planned : 10 total, 1 for each regular season starting from August 2008 -  April 2019
 
Dataset Explanation: This website contains datasets about the NBA teams’ statistics from year 1999 to present date. Those datasets are organized as in form of a table, where the rows represent the teams and the columns are observations such as win-loss record, shooting percentages, rebounds ..etc.  Since we are planning on using the statistic of the past 10 regular season matchups, this website is useful in that it contains the information from the previous years and we will have to select the options provided in the website (in the provided link) to get the optimal dataset for our project. We will try to pull the regular season information for each year (from 2008 to present date), then drop off teams that are not participating in the current year playoff. We might also need to differentiate each season statistics, perhaps giving more weight to recent year stats than older years.  


# Setup

In [1]:
# Imports 
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import bs4
from bs4 import BeautifulSoup
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, recall_score, f1_score

C:\Users\Kevin Mach\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#example URL
# https://www.basketball-reference.com/teams/MIL/2019/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8

year_array = ["2010", "2011", "2012", "2013", "2014", "2015", 
              "2016", "2017", "2018", "2019"]
team_array = ["GSW", "HOU", "POR", "DEN", "LAC", "UTA", "OKC", 
             "SAS", "MIL", "BOS", "PHI", "TOR", "DET", "IND", "ORL", "BRK"]

def create_url(year_array, team_array):
    #variables for appending the URLs
    begLink = "https://www.basketball-reference.com/teams/"
    endLink = "/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8"
    #array to store the url's
    url_list = []
    #loop through team_array 
    for i in team_array:
        #loop through each year in 
        for j in year_array:
            team_name = i
            # if the team is Brooklyn and the year is 2010-2012, team name
            # is New Jersey Nets
            if(i == "BRK" and (j == "2010" or j == "2011" or j == "2012")):
                team_name = "NJN"    
            temp_url = begLink + team_name + "/" + j + endLink
            #print(temp_url)
            url_list = np.append(url_list, temp_url)
    #print(url_list)
    return url_list

            
url_list = create_url(year_array, team_array)


In [3]:
url_list

array([ 'https://www.basketball-reference.com/teams/GSW/2010/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8',
       'https://www.basketball-reference.com/teams/GSW/2011/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8',
       'https://www.basketball-reference.com/teams/GSW/2012/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8',
       'https://www.basketball-reference.com/teams/GSW/2013/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8',
       'https://www.basketball-reference.com/teams/GSW/2014/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8',
       'https://www.basketball-reference.com/teams/GSW/2015/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8',
       'https://www.basketball-reference.com/teams/GSW/2016/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8',
       'https://www

In [4]:
#this code creates a list of dataframes corresponding to the url's in url_list

#list to store individual dataframes before combining
df_list = []
#i = 0
for url in url_list:
    #print(i)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser') #get contents of webpage
    nbatables = soup.findAll("table", 'row_summable sortable stats_table') #get tables
    tbl1 = nbatables[0]
    new_tbl1 = pd.DataFrame(columns=range(0,40), index = range(0,91))
    
    #get the column names for our first table
    ind=0
    cols_list = []
    for header in tbl1.find_all('tr'): #specify HTML tags
        header_name = header.find_all('th') #tag containing column names
        for head in header_name:
            cols_list.append(head.get_text()) #get the text from between the tags
    
    #fill in contents for each table
    row_marker = -1
    for row in tbl1.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td') # different tag than above for table contents
        for column in columns:
            new_tbl1.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        row_marker += 1
    df_list.append(new_tbl1)
    #i = i + 1

In [5]:
def create_team_df(team, df_list):
    #team_df = df_list[0]
    if(team == "GSW"):
        i = 0
        team_df = df_list[i];
        while i < 9:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "HOU"):
        i = 10
        team_df = df_list[i];
        while i < 19:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "POR"):
        i = 20
        team_df = df_list[i];
        while i < 29:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "DEN"):
        i = 30
        team_df = df_list[i];
        while i < 39:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "LAC"):
        i = 40
        team_df = df_list[i];
        while i < 49:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "UTA"):
        i = 50
        team_df = df_list[i];
        while i < 59:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "OKC"):
        i = 60
        team_df = df_list[i];
        while i < 69:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "SAS"):
        i = 70
        team_df = df_list[i];
        while i < 79:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "MIL"):
        i = 80
        team_df = df_list[i];
        while i < 89:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "BOS"):
        i = 90
        team_df = df_list[i];
        while i < 99:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "PHI"):
        i = 100
        team_df = df_list[i];
        while i < 109:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "TOR"):
        i = 110
        team_df = df_list[i];
        while i < 119:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "DET"):
        i = 120
        team_df = df_list[i];
        while i < 129:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "IND"):
        i = 130
        team_df = df_list[i];
        while i < 139:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "ORL"):
        i = 140
        team_df = df_list[i];
        while i < 149:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "BRK"):
        i = 150
        team_df = df_list[i];
        while i < 159:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    #NOTE: renamed opponent points to OppP
    team_df.columns = [
 'G',
 'Date',
 'Home',
 'Opp',
 'W/L',
 'Tm',
 'OppP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 '\xa0',
 'oppFG',
 'oppFGA',
 'oppFG%',
 'opp3P',
 'opp3PA',
 'opp3P%',
 'oppFT',
 'oppFTA',
 'oppFT%',
 'oppORB',
 'oppTRB',
 'oppAST',
 'oppSTL',
 'oppBLK',
 'oppTOV',
 'oppPF']
    return team_df

In [6]:
GSW_df = create_team_df("GSW", df_list)
HOU_df = create_team_df("HOU", df_list)
POR_df = create_team_df("POR", df_list)
DEN_df = create_team_df("DEN", df_list)
LAC_df = create_team_df("LAC", df_list)
UTA_df = create_team_df("UTA", df_list)
OKC_df = create_team_df("OKC", df_list)
SAS_df = create_team_df("SAS", df_list)
MIL_df = create_team_df("MIL", df_list)
BOS_df = create_team_df("BOS", df_list)
PHI_df = create_team_df("PHI", df_list)
TOR_df = create_team_df("TOR", df_list)
DET_df = create_team_df("DET", df_list)
IND_df = create_team_df("IND", df_list)
ORL_df = create_team_df("ORL", df_list)
BRK_df = create_team_df("BRK", df_list)

In [7]:
#Create a team dictionary to hold all of the dataframes. Key = team name, Value = team dataframe
teams_dict = {
    "GSW" : GSW_df,
    "HOU" : HOU_df,
    "POR" : POR_df,
    "DEN" : DEN_df,
    "LAC" : LAC_df,
    "UTA" : UTA_df,
    "OKC" : OKC_df,
    "SAS" : SAS_df,
    "MIL" : MIL_df,
    "BOS" : BOS_df,
    "PHI" : PHI_df,
    "TOR" : TOR_df,
    "DET" : DET_df,
    "IND" : IND_df,
    "ORL" : ORL_df,
    "BRK" : BRK_df
 } 

In [8]:
# Add the team as a column for each dataframe
for team, team_df in teams_dict.items():
    name = []
    # Creating a list filled with team's acronym
    for x in range(len(team_df.index)):
        name.append(team)
    team_df['Team'] = name

# Data Cleaning

Our end goal for the data cleaning would be that the resulting dataframes would have no nan values. In addition, some columns do not have any data that we think is important, such as the game #... etc, and we plan on removing those. Lastly, we want to transform some columns for easier reading and computing. In particular, the data set contains all of the game data of a particular team from year 2009 to 2019. However, a regular season will overlapped between two years, thus we want to change the date so that the year will be when the regular season ends. For the W/L record, the current data is displaying as a string "W" and "L" for win and loss. To make it easier for us to compute like the win-rate percent, we want the data to be in term of 1 and 0. The tools we will be using would be pandas library functions like dropna() as well as some of our own functions to transform the data.

In [9]:
#This array contains the month when the season begins
firstHalf   = ['10','11','12']

int_type = ['Tm', 'OppP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 
'oppFG', 'oppFGA', 'opp3P', 'opp3PA', 'oppFT', 'oppFTA', 'oppORB', 'oppTRB', 'oppAST', 'oppSTL', 'oppBLK', 'oppTOV','oppPF' ]

float_type = ['FG%', '3P%','FT%', 'oppFG%', 'opp3P%', 'oppFT%']



In [10]:
#This function allows us to filter out the date and month of the game. It is also easier to tell which year the data belongs to 
def standardize_year(input_year):
    year  = str(input_year)[:4]
    month = str(input_year)[5:7]
    if month in firstHalf:
        return int(year) + 1
    else:
        return int(year)

#This function allows us to transform the W/L to computable data. W => 1, L => 0    
def standardize_score(input_score):
    if input_score == "W":
        return int(1)
    else:
        return int(0)
    
def standardize_home(input_home):
    if input_home == "@":
        return int(1)
    else:
        return int(0)

In [11]:
#Cleaning the data
for w in team_array:
    #Getting dataframe of the current team
    currentTeam_df = teams_dict[w]      
    #Drop rows with nan values
    currentTeam_df = currentTeam_df.dropna() 
    #Drop columns that have unneccessary data 
    currentTeam_df = currentTeam_df.drop(labels = ['\xa0', 'G'], axis=1)            
    #Drop rows with teams that are not in the current 2019 playoff
    currentTeam_df = currentTeam_df[currentTeam_df['Opp'].isin(team_array)]
    #Filter and transform the "Date" column
    currentTeam_df['Date'] = currentTeam_df['Date'].apply(standardize_year)
    #Transform the "W/L" column 
    currentTeam_df['W/L']  = currentTeam_df['W/L'].apply(standardize_score)
    currentTeam_df['Home'] = currentTeam_df['Home'].apply(standardize_home)
    for label in int_type:
        currentTeam_df[label] = currentTeam_df[label].astype('int')
    for other_label in float_type:
        currentTeam_df[other_label] = currentTeam_df[other_label].astype('float')
    #Apply the modification
    teams_dict[w]  = currentTeam_df

# Data Analysis & Results

First we will compute the average game statistics for every team during the 2019 season against their respective opponent in the playoffs, as this will be used for the input for our model i.e. our "testing data". We will then concatenate these statistics to the original dataframe and begin feature engineering. Many of these features will consist of binary variables such as higherFG%, higherTOV, higher3P%, higherFT%, higherFGA, higherFTA, higher3PA, higherORB, higherTRB, higherBLK, higherSTL, higherAST, and higherPF. The label we wish to predict is whether or not they win the matchup.     

In [12]:
# Concatenating all the teams dataframes into one dataframe
df_all = pd.DataFrame()
for team_df in teams_dict.values():
    df_all = pd.concat([df_all, team_df], axis=0)

In [13]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6467 entries, 1 to 85
Data columns (total 39 columns):
Date      6467 non-null int64
Home      6467 non-null int64
Opp       6467 non-null object
W/L       6467 non-null int64
Tm        6467 non-null int32
OppP      6467 non-null int32
FG        6467 non-null int32
FGA       6467 non-null int32
FG%       6467 non-null float64
3P        6467 non-null int32
3PA       6467 non-null int32
3P%       6467 non-null float64
FT        6467 non-null int32
FTA       6467 non-null int32
FT%       6467 non-null float64
ORB       6467 non-null int32
TRB       6467 non-null int32
AST       6467 non-null int32
STL       6467 non-null int32
BLK       6467 non-null int32
TOV       6467 non-null int32
PF        6467 non-null int32
oppFG     6467 non-null int32
oppFGA    6467 non-null int32
oppFG%    6467 non-null float64
opp3P     6467 non-null int32
opp3PA    6467 non-null int32
opp3P%    6467 non-null float64
oppFT     6467 non-null int32
oppFTA    6467

In [14]:
# This function will calculate average matchup statistics between two teams in the 2019 season and append it to the overall dataframe
home_seeds = ["GSW", "DEN", "POR", "HOU", "MIL", "TOR", "PHI", "BOS"]

def average_stats_2019(team_name, df_team, df_total, opp):
    df_2019 = df_team[(df_team['Date'] == 2019) & (df_team['Opp'] == opp)]
    avgs = df_2019.mean(axis=0)
    avgs_df = avgs.to_frame().transpose()
    # Removing the 'W/L' column as that is what we will need to predict
    avgs_df.drop(labels = 'W/L', axis = 1, inplace = True)
    # Adding back the team and opp columns
    avgs_df['Team'] = team_name
    avgs_df['Opp'] = opp
    if (team_name in home_seeds):
        avgs_df['Home'] = 1
    else:
        avgs_df['Home'] = 0
    return pd.concat([df_total, avgs_df], axis = 0)

In [15]:
# Creating the average statistics between playoff matchups
for team, df_team in teams_dict.items():
    if (team == "GSW"):
        opponent = "LAC"
    elif (team == "HOU"):
        opponent = "UTA"
    elif (team == "PHI"):
        opponent = "BRK"
    elif (team == "UTA"):
        opponent = "HOU"
    elif (team == "MIL"):
        opponent = "DET"
    elif (team == "TOR"):
        opponent = "ORL"
    elif (team == "DET"):
        opponent = "MIL"
    elif (team == "BOS"):
        opponent = "IND"
    elif (team == "POR"):
        opponent = "OKC"
    elif (team == "ORL"):
        opponent = "TOR"
    elif (team == "LAC"):
        opponent = "GSW"
    elif (team == "IND"):
        opponent = "BOS"
    elif (team == "OKC"):
        opponent = "POR"
    elif (team == "BRK"):
        opponent = "PHI"
    elif (team == "DEN"):
        opponent = "SAS"
    elif (team == "SAS"):
        opponent = "DEN"
    else:
        continue        
    print(team + ' vs ' + opponent)
    new_df = average_stats_2019(team, df_team, df_all, opponent)
    df_all = new_df

PHI vs BRK
UTA vs HOU
MIL vs DET
GSW vs LAC
TOR vs ORL
DET vs MIL
BOS vs IND
POR vs OKC
ORL vs TOR
LAC vs GSW
IND vs BOS
OKC vs POR
HOU vs UTA
BRK vs PHI
DEN vs SAS
SAS vs DEN


In [16]:
# Checking out the data, delete later
df_all['Team'].tail(16)

0    PHI
0    UTA
0    MIL
0    GSW
0    TOR
0    DET
0    BOS
0    POR
0    ORL
0    LAC
0    IND
0    OKC
0    HOU
0    BRK
0    DEN
0    SAS
Name: Team, dtype: object

In [17]:
# Resetting index
df_all = df_all.reset_index()
df_all.drop(labels = 'index', axis = 1, inplace = True)

In [18]:
df_all.columns

Index([u'3P', u'3P%', u'3PA', u'AST', u'BLK', u'Date', u'FG', u'FG%', u'FGA',
       u'FT', u'FT%', u'FTA', u'Home', u'ORB', u'Opp', u'OppP', u'PF', u'STL',
       u'TOV', u'TRB', u'Team', u'Tm', u'W/L', u'opp3P', u'opp3P%', u'opp3PA',
       u'oppAST', u'oppBLK', u'oppFG', u'oppFG%', u'oppFGA', u'oppFT',
       u'oppFT%', u'oppFTA', u'oppORB', u'oppPF', u'oppSTL', u'oppTOV',
       u'oppTRB'],
      dtype='object')

### Feature Engineering

In [19]:
# higherFG%, higherTOV, higher3P%, higherFT%, higherFGA, higherFTA, higher3PA, higherORB, higherTRB, higherBLK, higherSTL 
# higherAST, higherPF, higherFGM, higher3PM, higherFTM, homecourt
train_data = pd.DataFrame()
train_data['W'] = df_all['W/L']
train_data['Team'] = df_all['Team']
train_data['Opp'] = df_all['Opp']
train_data['Home'] = df_all['Home']
train_data['higherFG%'] = (df_all['FG%'] - df_all['oppFG%']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherTOV'] = (df_all['TOV'] - df_all['oppTOV']).apply(lambda x: 0 if x < 0 else 1)
train_data['higher3P%'] = (df_all['3P%'] - df_all['opp3P%']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherFT%'] = (df_all['FT%'] - df_all['oppFT%']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherFGA'] = (df_all['FGA'] - df_all['oppFGA']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherFTA'] = (df_all['FTA'] - df_all['oppFTA']).apply(lambda x: 1 if x > 0 else 0)
train_data['higher3PA'] = (df_all['3PA'] - df_all['opp3PA']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherORB'] = (df_all['ORB'] - df_all['oppORB']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherTRB'] = (df_all['TRB'] - df_all['oppTRB']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherBLK'] = (df_all['BLK'] - df_all['oppBLK']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherSTL'] = (df_all['STL'] - df_all['oppSTL']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherAST'] = (df_all['AST'] - df_all['oppAST']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherPF'] = (df_all['PF'] - df_all['oppPF']).apply(lambda x: 0 if x > 0 else 1)
train_data['higherFGM'] = (df_all['FG'] - df_all['oppFG']).apply(lambda x: 1 if x > 0 else 0)
train_data['higher3PM'] = (df_all['3P'] - df_all['opp3P']).apply(lambda x: 1 if x > 0 else 0)
train_data['higherFTM'] = (df_all['FT'] - df_all['oppFT']).apply(lambda x: 1 if x > 0 else 0)

In [20]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6483 entries, 0 to 6482
Data columns (total 20 columns):
W            6467 non-null float64
Team         6483 non-null object
Opp          6483 non-null object
Home         6483 non-null int64
higherFG%    6483 non-null int64
higherTOV    6483 non-null int64
higher3P%    6483 non-null int64
higherFT%    6483 non-null int64
higherFGA    6483 non-null int64
higherFTA    6483 non-null int64
higher3PA    6483 non-null int64
higherORB    6483 non-null int64
higherTRB    6483 non-null int64
higherBLK    6483 non-null int64
higherSTL    6483 non-null int64
higherAST    6483 non-null int64
higherPF     6483 non-null int64
higherFGM    6483 non-null int64
higher3PM    6483 non-null int64
higherFTM    6483 non-null int64
dtypes: float64(1), int64(17), object(2)
memory usage: 1013.0+ KB


In [21]:
train_data.tail(16)

,W,Team,Opp,Home,higherFG%,higherTOV,higher3P%,higherFT%,higherFGA,higherFTA,higher3PA,higherORB,higherTRB,higherBLK,higherSTL,higherAST,higherPF,higherFGM,higher3PM,higherFTM
6467,NaN,PHI,BRK,1,1,1,0,1,0,1,0,0,1,1,1,1,1,0,0,1
6468,NaN,UTA,HOU,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,0,1
6469,NaN,MIL,DET,1,1,0,1,0,0,1,0,0,0,1,1,1,1,1,0,1
6470,NaN,GSW,LAC,1,1,0,0,1,1,0,1,1,1,1,1,1,1,1,1,0
6471,NaN,TOR,ORL,1,0,1,1,1,0,0,1,0,0,0,1,0,0,0,1,0
6472,NaN,DET,MIL,0,0,1,0,1,1,0,1,1,1,0,0,0,0,0,1,0
6473,NaN,BOS,IND,1,1,1,1,1,0,0,1,0,1,1,0,1,0,1,1,0
6474,NaN,POR,OKC,1,0,1,0,1,1,1,1,1,1,1,0,1,1,0,0,1
6475,NaN,ORL,TOR,0,1,1,0,0,1,1,0,1,1,1,0,1,1,1,0,1
6476,NaN,LAC,GSW,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1


In [22]:
# Splitting the data into the training set and testing set
training_data = train_data.iloc[0:6467, :]
testing_data = train_data.iloc[6467: , :]

In [23]:
# Splitting training data into x and y
x = training_data.drop(labels = ['W', 'Team', 'Opp'], axis = 1)
y = training_data['W']

In [24]:
# Train and test split
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=0)
print X_train.shape, X_test.shape, y_train.shape, y_test.shape

(5173, 17) (1294, 17) (5173L,) (1294L,)


In [25]:
# Regression Technique
lr = LogisticRegression()
logmodel = lr.fit(X_train, y_train)

In [26]:
# Prediction
prediction = logmodel.predict(X_test)

In [27]:
# Checking accuracy of our model
accuracy_score(y_test, prediction)

0.88871715610510049

### Playoff Predictions
#### Round 1

In [28]:
x1 = testing_data.drop(labels = ['W', 'Team', 'Opp'], axis = 1)
round1_predictions = logmodel.predict(x1)

In [29]:
# Formating into a dataframe
round1_results = pd.DataFrame()
round1_results['Team'] = testing_data['Team']
round1_results['Opp'] = testing_data['Opp']
round1_results['W'] = round1_predictions.astype(int)

round1_results

,Team,Opp,W
6467,PHI,BRK,1
6468,UTA,HOU,1
6469,MIL,DET,1
6470,GSW,LAC,1
6471,TOR,ORL,0
6472,DET,MIL,0
6473,BOS,IND,1
6474,POR,OKC,0
6475,ORL,TOR,1
6476,LAC,GSW,0


In [30]:
# Selecting the winners from the dataframe
round1_winners = []
for ind in round1_results.index:
    if (round1_results['W'][ind] == 1):
        round1_winners.append(round1_results['Team'][ind])
    else:
        continue
print(round1_winners)

['PHI', 'UTA', 'MIL', 'GSW', 'BOS', 'ORL', 'OKC', 'DEN']


### Round 2

In [31]:
# Creating the average statistics between playoff matchups for the second round
round2avgs = pd.DataFrame()

for team in round1_winners:
    if (team == "GSW"):
        if ("HOU" in round1_winners):
            opponent = "HOU"
        else:
            opponent = "UTA"
    elif (team == "HOU"):
        if ("GSW" in round1_winners):
            opponent = "GSW"
        else:
            opponent = "LAC"
    elif (team == "PHI"):
        if ("TOR" in round1_winners):
            opponent = "TOR"
        else:
            opponent = "ORL"
    elif (team == "UTA"):
        if ("GSW" in round1_winners):
            opponent = "GSW"
        else:
            opponent = "LAC"
    elif (team == "MIL"):
        if ("BOS" in round1_winners):
            opponent = "BOS"
        else:
            opponent = "IND"
    elif (team == "TOR"):
        if ("PHI" in round1_winners):
            opponent = "PHI"
        else:
            opponent = "BRK"
    elif (team == "DET"):
        if ("BOS" in round1_winners):
            opponent = "BOS"
        else:
            opponent = "IND"
    elif (team == "BOS"):
        if ("MIL" in round1_winners):
            opponent = "MIL"
        else:
            opponent = "DET"
    elif (team == "POR"):
        if ("DEN" in round1_winners):
            opponent = "DEN"
        else:
            opponent = "SAS"
    elif (team == "ORL"):
        if ("PHI" in round1_winners):
            opponent = "PHI"
        else:
            opponent = "BRK"
    elif (team == "LAC"):
        if ("HOU" in round1_winners):
            opponent = "HOU"
        else:
            opponent = "UTA"
    elif (team == "IND"):
        if ("MIL" in round1_winners):
            opponent = "MIL"
        else:
            opponent = "DET"
    elif (team == "OKC"):
        if ("DEN" in round1_winners):
            opponent = "DEN"
        else:
            opponent = "SAS"
    elif (team == "BRK"):
        if ("TOR" in round1_winners):
            opponent = "TOR"
        else:
            opponent = "ORL"
    elif (team == "DEN"):
        if ("POR" in round1_winners):
            opponent = "POR"
        else:
            opponent = "OKC"
    elif (team == "SAS"):
        if ("POR" in round1_winners):
            opponent = "POR"
        else:
            opponent = "OKC"
    else:
        continue        
    print(team + ' vs ' + opponent)
    new_df = average_stats_2019(team, teams_dict[team], round2avgs, opponent)
    round2avgs = new_df

PHI vs ORL
UTA vs GSW
MIL vs BOS
GSW vs UTA
BOS vs MIL
ORL vs PHI
OKC vs DEN
DEN vs OKC


In [32]:
round2avgs.head()

,Date,Home,Tm,OppP,FG,FGA,FG%,3P,3PA,3P%,...,oppFT%,oppORB,oppTRB,oppAST,oppSTL,oppBLK,oppTOV,oppPF,Team,Opp
0,2019.0,1,108.500000,112.750000,41.000000,85.500000,0.480250,12.0,28.250000,0.413000,...,0.792500,10.500000,43.500000,25.750000,6.750000,3.750000,10.750000,17.500000,PHI,ORL
0,2019.0,0,113.000000,114.000000,39.333333,89.666667,0.439333,15.0,38.000000,0.393000,...,0.833333,7.666667,43.666667,24.666667,10.666667,7.333333,11.333333,21.333333,UTA,GSW
0,2019.0,1,110.333333,107.000000,39.666667,84.333333,0.471667,13.0,36.000000,0.356333,...,0.809667,9.666667,42.666667,23.666667,9.666667,6.666667,8.000000,18.000000,MIL,BOS
0,2019.0,1,114.000000,113.000000,43.666667,90.333333,0.485667,11.0,30.333333,0.388667,...,0.802667,9.000000,46.333333,25.666667,7.000000,4.666667,16.000000,18.666667,GSW,UTA
0,2019.0,1,107.000000,110.333333,38.333333,95.000000,0.404333,16.0,43.000000,0.360000,...,0.744333,9.333333,51.666667,22.333333,5.333333,4.333333,16.333333,20.333333,BOS,MIL


In [33]:
# Feature engineering for the new dataframe
# higherFG%, higherTOV, higher3P%, higherFT%, higherFGA, higherFTA, higher3PA, higherORB, higherTRB, higherBLK, higherSTL 
# higherAST, higherPF, higherFGM, higher3PM, higherFTM

testing_data2 = pd.DataFrame()
testing_data2['Team'] = round2avgs['Team']
testing_data2['Opp'] = round2avgs['Opp']
testing_data2['Home'] = round2avgs['Home']
testing_data2['higherFG%'] = (round2avgs['FG%'] - round2avgs['oppFG%']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherTOV'] = (round2avgs['TOV'] - round2avgs['oppTOV']).apply(lambda x: 0 if x < 0 else 1)
testing_data2['higher3P%'] = (round2avgs['3P%'] - round2avgs['opp3P%']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherFT%'] = (round2avgs['FT%'] - round2avgs['oppFT%']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherFGA'] = (round2avgs['FGA'] - round2avgs['oppFGA']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherFTA'] = (round2avgs['FTA'] - round2avgs['oppFTA']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higher3PA'] = (round2avgs['3PA'] - round2avgs['opp3PA']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherORB'] = (round2avgs['ORB'] - round2avgs['oppORB']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherTRB'] = (round2avgs['TRB'] - round2avgs['oppTRB']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherBLK'] = (round2avgs['BLK'] - round2avgs['oppBLK']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherSTL'] = (round2avgs['STL'] - round2avgs['oppSTL']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherAST'] = (round2avgs['AST'] - round2avgs['oppAST']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherPF'] = (round2avgs['PF'] - round2avgs['oppPF']).apply(lambda x: 0 if x > 0 else 1)
testing_data2['higherFGM'] = (round2avgs['FG'] - round2avgs['oppFG']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higher3PM'] = (round2avgs['3P'] - round2avgs['opp3P']).apply(lambda x: 1 if x > 0 else 0)
testing_data2['higherFTM'] = (round2avgs['FT'] - round2avgs['oppFT']).apply(lambda x: 1 if x > 0 else 0)

In [34]:
testing_data2

,Team,Opp,Home,higherFG%,higherTOV,higher3P%,higherFT%,higherFGA,higherFTA,higher3PA,higherORB,higherTRB,higherBLK,higherSTL,higherAST,higherPF,higherFGM,higher3PM,higherFTM
0,PHI,ORL,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0
0,UTA,GSW,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,1,1
0,MIL,BOS,1,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1
0,GSW,UTA,1,1,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0
0,BOS,MIL,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,0
0,ORL,PHI,0,1,0,1,0,1,1,1,1,0,1,1,0,1,1,1,1
0,OKC,DEN,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,0
0,DEN,OKC,1,1,1,1,1,0,0,1,0,1,1,0,1,0,1,1,1


### Playoff Predictions
#### Round 2

In [35]:
# Using our model to predict round 2 winners
x2 = testing_data2.drop(labels = ['Team', 'Opp'], axis = 1)
round2_predictions = logmodel.predict(x2)

In [36]:
# Formating into a dataframe
round2_results = pd.DataFrame()
round2_results['Team'] = testing_data2['Team']
round2_results['Opp'] = testing_data2['Opp']
round2_results['W'] = round2_predictions.astype(int)

In [37]:
# Resetting index to loop through the df properly
round2_results = round2_results.reset_index()
round2_results.drop(labels = 'index', axis = 1, inplace = True)
round2_results

,Team,Opp,W
0,PHI,ORL,0
1,UTA,GSW,1
2,MIL,BOS,1
3,GSW,UTA,0
4,BOS,MIL,0
5,ORL,PHI,1
6,OKC,DEN,0
7,DEN,OKC,1


In [38]:
# Selecting the winners from the dataframe
round2_winners = []
for ind in round2_results.index:
    if (round2_results['W'][ind] == 1):
        round2_winners.append(round2_results['Team'][ind])
    else:
        continue
print(round2_winners)

['UTA', 'MIL', 'ORL', 'DEN']


### Round 3

In [39]:
# Creating the average statistics between playoff matchups for the third round
round3avgs = pd.DataFrame()

for team in round2_winners:
    if (team == "GSW"):
        for opp in round2_winners:
            if (opp in ["POR", "OKC", "DEN", "SAS"]):
                opponent = opp
    elif (team == "HOU"):
        for opp in round2_winners:
            if (opp in ["POR", "OKC", "DEN", "SAS"]):
                opponent = opp
    elif (team == "PHI"):
        for opp in round2_winners:
            if (opp in ["MIL", "DET", "BOS", "IND"]):
                opponent = opp
    elif (team == "UTA"):
        for opp in round2_winners:
            if (opp in ["POR", "OKC", "DEN", "SAS"]):
                opponent = opp
    elif (team == "MIL"):
        for opp in round2_winners:
            if (opp in ["PHI", "BRK", "TOR", "ORL"]):
                opponent = opp
    elif (team == "TOR"):
        for opp in round2_winners:
            if (opp in ["MIL", "DET", "BOS", "IND"]):
                opponent = opp
    elif (team == "DET"):
        for opp in round2_winners:
            if (opp in ["PHI", "BRK", "TOR", "ORL"]):
                opponent = opp
    elif (team == "BOS"):
        for opp in round2_winners:
            if (opp in ["PHI", "BRK", "TOR", "ORL"]):
                opponent = opp
    elif (team == "POR"):
        for opp in round2_winners:
            if (opp in ["GSW", "LAC", "HOU", "UTA"]):
                opponent = opp
    elif (team == "ORL"):
        for opp in round2_winners:
            if (opp in ["MIL", "DET", "BOS", "IND"]):
                opponent = opp
    elif (team == "LAC"):
        for opp in round2_winners:
            if (opp in ["POR", "OKC", "DEN", "SAS"]):
                opponent = opp
    elif (team == "IND"):
        for opp in round2_winners:
            if (opp in ["PHI", "BRK", "TOR", "ORL"]):
                opponent = opp
    elif (team == "OKC"):
        for opp in round2_winners:
            if (opp in ["GSW", "LAC", "HOU", "UTA"]):
                opponent = opp
    elif (team == "BRK"):
        for opp in round2_winners:
            if (opp in ["MIL", "DET", "BOS", "IND"]):
                opponent = opp
    elif (team == "DEN"):
        for opp in round2_winners:
            if (opp in ["GSW", "LAC", "HOU", "UTA"]):
                opponent = opp
    elif (team == "SAS"):
        for opp in round2_winners:
            if (opp in ["GSW", "LAC", "HOU", "UTA"]):
                opponent = opp
    else:
        continue        
    print(team + ' vs ' + opponent)
    new_df = average_stats_2019(team, teams_dict[team], round3avgs, opponent)
    round3avgs = new_df

UTA vs DEN
MIL vs ORL
ORL vs MIL
DEN vs UTA


In [40]:
# Feature engineering for the new dataframe
# higherFG%, higherTOV, higher3P%, higherFT%, higherFGA, higherFTA, higher3PA, higherORB, higherTRB, higherBLK, higherSTL 
# higherAST, higherPF, higherFGM, higher3PM, higherFTM

testing_data3 = pd.DataFrame()
testing_data3['Team'] = round3avgs['Team']
testing_data3['Opp'] = round3avgs['Opp']
testing_data3['Home'] = round3avgs['Home']
testing_data3['higherFG%'] = (round3avgs['FG%'] - round3avgs['oppFG%']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherTOV'] = (round3avgs['TOV'] - round3avgs['oppTOV']).apply(lambda x: 0 if x < 0 else 1)
testing_data3['higher3P%'] = (round3avgs['3P%'] - round3avgs['opp3P%']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherFT%'] = (round3avgs['FT%'] - round3avgs['oppFT%']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherFGA'] = (round3avgs['FGA'] - round3avgs['oppFGA']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherFTA'] = (round3avgs['FTA'] - round3avgs['oppFTA']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higher3PA'] = (round3avgs['3PA'] - round3avgs['opp3PA']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherORB'] = (round3avgs['ORB'] - round3avgs['oppORB']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherTRB'] = (round3avgs['TRB'] - round3avgs['oppTRB']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherBLK'] = (round3avgs['BLK'] - round3avgs['oppBLK']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherSTL'] = (round3avgs['STL'] - round3avgs['oppSTL']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherAST'] = (round3avgs['AST'] - round3avgs['oppAST']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherPF'] = (round3avgs['PF'] - round3avgs['oppPF']).apply(lambda x: 0 if x > 0 else 1)
testing_data3['higherFGM'] = (round3avgs['FG'] - round3avgs['oppFG']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higher3PM'] = (round3avgs['3P'] - round3avgs['opp3P']).apply(lambda x: 1 if x > 0 else 0)
testing_data3['higherFTM'] = (round3avgs['FT'] - round3avgs['oppFT']).apply(lambda x: 1 if x > 0 else 0)

In [41]:
testing_data3

,Team,Opp,Home,higherFG%,higherTOV,higher3P%,higherFT%,higherFGA,higherFTA,higher3PA,higherORB,higherTRB,higherBLK,higherSTL,higherAST,higherPF,higherFGM,higher3PM,higherFTM
0,UTA,DEN,0,0,1,0,0,0,1,1,1,0,1,0,0,1,0,1,1
0,MIL,ORL,1,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,1
0,ORL,MIL,0,0,0,1,0,1,0,1,1,0,0,1,1,0,1,1,0
0,DEN,UTA,1,1,0,1,1,1,0,0,0,1,0,0,1,0,1,0,0


### Playoff Predictions
#### Round 3

In [42]:
# Using our model to predict round 3 winners
x3 = testing_data3.drop(labels = ['Team', 'Opp'], axis = 1)
round3_predictions = logmodel.predict(x3)

In [43]:
# Formating into a dataframe
round3_results = pd.DataFrame()
round3_results['Team'] = testing_data3['Team']
round3_results['Opp'] = testing_data3['Opp']
round3_results['W'] = round3_predictions.astype(int)

round3_results

,Team,Opp,W
0,UTA,DEN,0
0,MIL,ORL,1
0,ORL,MIL,1
0,DEN,UTA,1


In [44]:
# Resetting index to loop through the df properly
round3_results = round3_results.reset_index()
round3_results.drop(labels = 'index', axis = 1, inplace = True)
round3_results

,Team,Opp,W
0,UTA,DEN,0
1,MIL,ORL,1
2,ORL,MIL,1
3,DEN,UTA,1


In [45]:
# Selecting the winners from the dataframe
round3_winners = []
for ind in round3_results.index:
    if (round3_results['W'][ind] == 1):
        round3_winners.append(round3_results['Team'][ind])
    else:
        continue
print(round3_winners)

['MIL', 'ORL', 'DEN']


In [46]:
# We will give the higher seeded team the win if our model predicts that both teams win
round3_winners.remove('ORL')
round3_winners

['MIL', 'DEN']

### Finals

In [47]:
# Creating the average statistics between playoff matchups for the third round
round4avgs = pd.DataFrame()
team1 = round3_winners[0]
team2 = round3_winners[1]
df1 = average_stats_2019(team1, teams_dict[team1], round4avgs, team2)
round4avgs = df1
df2 = average_stats_2019(team2, teams_dict[team2], round4avgs, team1)
round4avgs = df2

In [48]:
round4avgs

,Date,Home,Tm,OppP,FG,FGA,FG%,3P,3PA,3P%,...,oppFT%,oppORB,oppTRB,oppAST,oppSTL,oppBLK,oppTOV,oppPF,Team,Opp
0,2019.0,1,112.5,106.0,44.0,86.0,0.5130,14.0,35.0,0.4030,...,0.6835,11.5,41.0,28.5,7.0,4.5,13.5,18.0,MIL,DEN
0,2019.0,1,106.0,112.5,40.5,91.5,0.4435,15.0,39.5,0.3835,...,0.6030,9.0,46.0,24.0,9.0,5.0,13.5,16.5,DEN,MIL


In [49]:
# As MIL had the better regular season record, they get homecourt advantage
round4avgs['Home'] = [1,0]

In [50]:
# Feature engineering for the new dataframe
# higherFG%, higherTOV, higher3P%, higherFT%, higherFGA, higherFTA, higher3PA, higherORB, higherTRB, higherBLK, higherSTL 
# higherAST, higherPF, higherFGM, higher3PM, higherFTM

testing_data4 = pd.DataFrame()
testing_data4['Team'] = round4avgs['Team']
testing_data4['Opp'] = round4avgs['Opp']
testing_data4['Home'] = round4avgs['Home']
testing_data4['higherFG%'] = (round4avgs['FG%'] - round4avgs['oppFG%']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherTOV'] = (round4avgs['TOV'] - round4avgs['oppTOV']).apply(lambda x: 0 if x < 0 else 1)
testing_data4['higher3P%'] = (round4avgs['3P%'] - round4avgs['opp3P%']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherFT%'] = (round4avgs['FT%'] - round4avgs['oppFT%']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherFGA'] = (round4avgs['FGA'] - round4avgs['oppFGA']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherFTA'] = (round4avgs['FTA'] - round4avgs['oppFTA']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higher3PA'] = (round4avgs['3PA'] - round4avgs['opp3PA']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherORB'] = (round4avgs['ORB'] - round4avgs['oppORB']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherTRB'] = (round4avgs['TRB'] - round4avgs['oppTRB']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherBLK'] = (round4avgs['BLK'] - round4avgs['oppBLK']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherSTL'] = (round4avgs['STL'] - round4avgs['oppSTL']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherAST'] = (round4avgs['AST'] - round4avgs['oppAST']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherPF'] = (round4avgs['PF'] - round4avgs['oppPF']).apply(lambda x: 0 if x > 0 else 1)
testing_data4['higherFGM'] = (round4avgs['FG'] - round4avgs['oppFG']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higher3PM'] = (round4avgs['3P'] - round4avgs['opp3P']).apply(lambda x: 1 if x > 0 else 0)
testing_data4['higherFTM'] = (round4avgs['FT'] - round4avgs['oppFT']).apply(lambda x: 1 if x > 0 else 0)

In [51]:
testing_data4

,Team,Opp,Home,higherFG%,higherTOV,higher3P%,higherFT%,higherFGA,higherFTA,higher3PA,higherORB,higherTRB,higherBLK,higherSTL,higherAST,higherPF,higherFGM,higher3PM,higherFTM
0,MIL,DEN,1,1,1,1,0,0,1,0,0,1,1,1,0,1,1,0,1
0,DEN,MIL,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,1,0


### Playoff Predictions
#### Finals

In [52]:
# Using our model to predict champion
x4 = testing_data4.drop(labels = ['Team', 'Opp'], axis = 1)
round4_predictions = logmodel.predict(x4)

In [53]:
# Formating into a dataframe
round4_results = pd.DataFrame()
round4_results['Team'] = testing_data4['Team']
round4_results['Opp'] = testing_data4['Opp']
round4_results['W'] = round4_predictions.astype(int)

round4_results

,Team,Opp,W
0,MIL,DEN,1
0,DEN,MIL,0


In [54]:
# Resetting index to loop through the df properly
round4_results = round4_results.reset_index()
round4_results.drop(labels = 'index', axis = 1, inplace = True)
round4_results

,Team,Opp,W
0,MIL,DEN,1
1,DEN,MIL,0


In [55]:
# Selecting the winners from the dataframe
champ = []
for ind in round4_results.index:
    if (round4_results['W'][ind] == 1):
        champ.append(round4_results['Team'][ind])
    else:
        continue
print(champ)

['MIL']


# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*